# 浪子之心科技
# 此代码仅为演示wav2lip推理结束后用CodeFormer进行高清化的过程。
# 效果比Wav2Lip-GFPGAN好。


In [ ]:
#@title 一、下载wav2lip源码和权重并安装wav2lip环境
#@title 
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/zabique/Wav2Lip

#download the pretrained model
!wget 'https://iiitaphyd-my.sharepoint.com/personal/radrabha_m_research_iiit_ac_in/_layouts/15/download.aspx?share=EdjI7bZlgApMqsVoEUUXpLsBxqXbn5z8VTmoxp55YNDcIA' -O '/content/Wav2Lip/checkpoints/wav2lip_gan.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl

# !pip uninstall tensorflow tensorflow-gpu
!cd Wav2Lip && pip install -r requirements.txt

#download pretrained model for face detection
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

!pip install -q youtube-dl
!pip install ffmpeg-python
!pip install librosa==0.9.1

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

from IPython.display import clear_output 
clear_output()
print("\nDone")


Done


In [ ]:
#@title 二、下载CodeFormer的源码权重并安装依赖

# 下载CodeFormer的源码
%cd /content
!rm -rf CodeFormer
!git clone https://github.com/sczhou/CodeFormer.git
%cd /content/CodeFormer

/content
Cloning into 'CodeFormer'...
remote: Enumerating objects: 583, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 583 (delta 167), reused 166 (delta 144), pack-reused 349
Receiving objects: 100% (583/583), 17.31 MiB | 21.33 MiB/s, done.
Resolving deltas: 100% (278/278), done.
/content/CodeFormer


In [ ]:
# 下载下载CodeFormer的权重
%cd /content/CodeFormer
!python scripts/download_pretrained_models.py facelib
!python scripts/download_pretrained_models.py CodeFormer

/content/CodeFormer
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/CodeFormer/weights/facelib/detection_Resnet50_Final.pth

100% 104M/104M [00:00<00:00, 298MB/s] 
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/parsing_parsenet.pth" to /content/CodeFormer/weights/facelib/parsing_parsenet.pth

100% 81.4M/81.4M [00:00<00:00, 278MB/s]
Downloading: "https://github.com/sczhou/CodeFormer/releases/download/v0.1.0/codeformer.pth" to /content/CodeFormer/weights/CodeFormer/codeformer.pth

100% 359M/359M [00:02<00:00, 167MB/s]


In [ ]:
# 安装CodeFormer的依赖
%cd /content/CodeFormer
!python basicsr/setup.py develop

/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
 

In [ ]:
#@title 三、上传视频和音频
# 创建相关的文件夹
%cd /content
!mkdir output 
!mkdir input 
# 上传音视频
%cd /content/input 
from google.colab import files
from io import BytesIO
uploaded = files.upload()

/content


In [ ]:
#@title 四、用wav2lip初步生成修改嘴型后的视频

# 先清理之前生成的临时文件
%cd /content
!rm -rf output
!mkdir output 
!rm -rf /content/Wav2Lip/temp/result.avi
!rm -rf /content/Wav2Lip/temp/temp.wav
!rm -rf /content/Wav2Lip/results/result_voice.mp4
!rm -rf images
!mkdir images 
%cd /content/CodeFormer
!rm -rf results
!mkdir results 

/content
/content/CodeFormer


In [ ]:
# 用wav2lip进行推理
# checkpoints 可以用wav2lip.pth 或者wav2lip_gan.pt
%cd /content/Wav2Lip 
!python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face '/content/input/1.mp4' --audio '/content/input/1.mp3'

/content/Wav2Lip
Using cuda for inference.
Reading video frames...
Number of frames available for inference: 270
Extracting raw audio...
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsn

In [ ]:
# 将Wav2Lip生成的视频转化成图片保存到images
%cd /content
!mkdir images
%cd /content/Wav2Lip
import cv2
from tqdm import tqdm
from os import path
import os

vidcap = cv2.VideoCapture("/content/Wav2Lip/results/result_voice.mp4")
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _,image = vidcap.read()
    cv2.imwrite(path.join('/content/images', str(frameNumber).zfill(4)+'.jpg'), image)

/content
mkdir: cannot create directory ‘images’: File exists
/content/Wav2Lip
FPS:  30.0 Frames:  222


100%|██████████| 222/222 [00:05<00:00, 37.21it/s]


In [ ]:
#@title 五、用CodeFormer进行人脸修复

%cd /content/CodeFormer
# 定义相关函数
import cv2
import matplotlib.pyplot as plt
def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('CodeFormer', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)
def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

/content/CodeFormer


In [ ]:
# 用CodeFormer进行推理
%cd /content/CodeFormer
## 只修复人脸
# CODEFORMER_FIDELITY = 0.5  
# !python inference_codeformer.py -w $CODEFORMER_FIDELITY --has_aligned --input_path /content/images

# whole images ##整体图片修复
CODEFORMER_FIDELITY = 0.7
!python inference_codeformer.py -w $CODEFORMER_FIDELITY --input_path /content/images --bg_upsampler realesrgan

/content/CodeFormer
Face detection model: retinaface_resnet50
Background upsampling: True, Face upsampling: False
[1/222] Processing: 0000.jpg
	detect 1 faces
[2/222] Processing: 0001.jpg
	detect 1 faces
[3/222] Processing: 0002.jpg
	detect 1 faces
[4/222] Processing: 0003.jpg
	detect 1 faces
[5/222] Processing: 0004.jpg
	detect 1 faces
[6/222] Processing: 0005.jpg
	detect 1 faces
[7/222] Processing: 0006.jpg
	detect 1 faces
[8/222] Processing: 0007.jpg
	detect 1 faces
[9/222] Processing: 0008.jpg
	detect 1 faces
[10/222] Processing: 0009.jpg
	detect 1 faces
[11/222] Processing: 0010.jpg
	detect 1 faces
[12/222] Processing: 0011.jpg
	detect 1 faces
[13/222] Processing: 0012.jpg
	detect 1 faces
[14/222] Processing: 0013.jpg
	detect 1 faces
[15/222] Processing: 0014.jpg
	detect 1 faces
[16/222] Processing: 0015.jpg
	detect 1 faces
[17/222] Processing: 0016.jpg
	detect 1 faces
[18/222] Processing: 0017.jpg
	detect 1 faces
[19/222] Processing: 0018.jpg
	detect 1 faces
[20/222] Processing: 

In [ ]:
# 生成修复后的新视频
import os
restoredFramesPath = '/content/CodeFormer/results/images_0.7' + '/final_results/'
processedVideoOutputPath = '/content/output'

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()

import cv2
import numpy as np

batch = 0
batchSize = 600
from tqdm import tqdm
for i in tqdm(range(0, len(dir_list), batchSize)):
  img_array = []
  start, end = i, i+batchSize
  print("processing ", start, end)
  for filename in  tqdm(dir_list[start:end]):
      filename = restoredFramesPath+filename;
      img = cv2.imread(filename)
      if img is None:
        continue
      height, width, layers = img.shape
      size = (width,height)
      img_array.append(img)


  out = cv2.VideoWriter(processedVideoOutputPath+'/batch_'+str(batch).zfill(4)+'.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
  batch = batch + 1
 
  for i in range(len(img_array)):
    out.write(img_array[i])
  out.release()

# 音视频合并 Generate audio video
audio = ffmpeg.input(f'/content/input/5566.mp3')
video = ffmpeg.input(f'/content/output/batch_0000.avi')
print("合并视音频")
out = ffmpeg.output(video, audio, f'/content/output/final.mp4')
out.run()  
print("恭喜您，音视频合并完成，存放在output/final.mp4")

  0%|          | 0/1 [00:00<?, ?it/s]

processing  0 600



100%|██████████| 1/1 [00:23<00:00, 23.04s/it]


合并视音频
恭喜您，音视频合并完成，存放在output/final.mp4
